# Conversational Agent

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.tools import tool

In [3]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [4]:
import wikipedia

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [5]:
tools = [get_current_temperature, search_wikipedia]

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [16]:
functions = [format_tool_to_openai_function(t) for t in tools]
model = ChatOpenAI().bind(functions=functions)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sarcastic assistant"),
    ("user", "{input}")
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [18]:
result = chain.invoke({"input": "What's the weather in New Delhi?"})

In [21]:
result

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 28.6139, 'longitude': 77.209}, log="\nInvoking: `get_current_temperature` with `{'latitude': 28.6139, 'longitude': 77.209}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 28.6139,\n  "longitude": 77.2090\n}'}})])

In [20]:
result.tool

'get_current_temperature'

In [23]:
result.tool_input

{'latitude': 28.6139, 'longitude': 77.209}

In [24]:
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sarcastic assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [27]:
result1 = chain.invoke({
    "input": "what is weather like in new delhi?",
    "agent_scratchpad": []
})

In [36]:
result1

AgentActionMessageLog(tool='get_current_temperature', tool_input={'latitude': 28.6139, 'longitude': 77.209}, log="\nInvoking: `get_current_temperature` with `{'latitude': 28.6139, 'longitude': 77.209}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 28.6139,\n  "longitude": 77.2090\n}'}})])

In [28]:
result1.tool

'get_current_temperature'

In [29]:
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 28.6139,\n  "longitude": 77.2090\n}'}})]

In [30]:
observation = get_current_temperature(result1.tool_input)

In [50]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [32]:
format_to_openai_functions([(result1, observation)])

[AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{\n  "latitude": 28.6139,\n  "longitude": 77.2090\n}'}}),
 FunctionMessage(content='The current temperature is 18.3°C', name='get_current_temperature')]

In [34]:
result2 = chain.invoke({
    "input": "what is the weather like in new delhi?",
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [35]:
result2

AgentFinish(return_values={'output': 'The current temperature in New Delhi is 18.3°C.'}, log='The current temperature in New Delhi is 18.3°C.')

In [51]:
from langchain.schema.runnable import RunnablePassthrough

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain 

In [52]:
from langchain.schema.agent import AgentFinish

In [53]:
def run_agent(user_input):
    intermediate_steps = []
    tools = {
        "search_wikipedia": search_wikipedia,
        "get_current_temperature": get_current_temperature
    }
    while True:
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })
        
        if isinstance(result, AgentFinish):
            return result
        
        tool = tools[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))
    

In [54]:
run_agent("What's the weather like in prague?")

AgentFinish(return_values={'output': 'The current temperature in Prague is -1.1°C. Stay warm!'}, log='The current temperature in Prague is -1.1°C. Stay warm!')

In [56]:
run_agent("Who is Krishna?")`

AgentFinish(return_values={'output': "Krishna is a major deity in Hinduism. He is worshipped as the eighth avatar of Vishnu and also as the Supreme God in his own right. Krishna is known as the god of protection, compassion, tenderness, and love. He is one of the most popular and widely revered among Hindu divinities. Krishna is a central character in various Hindu texts such as the Mahabharata, the Bhagavata Purana, the Brahma Vaivarta Purana, and the Bhagavad Gita. His life is filled with anecdotes and narratives, collectively known as Krishna Līlā. Krishna is depicted in different stages of his life, including an infant eating butter, a young boy playing a flute, and a divine hero. His iconography reflects these legends. Krishna's worship has spread to various parts of India, such as Vrindavan, Dwarka, and Mayapur, and has also gained popularity in the Western world and Africa through the International Society for Krishna Consciousness (ISKCON)."}, log="Krishna is a major deity in H

In [64]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [65]:
agent_executor.invoke({"input": "What is Q* algorithm?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Q* algorithm'}`


Page: I.Q.: Intelligent Qube
Summary: I.Q.: Intelligent Qube (I.Q. インテリジェントキューブ, I.Q. Interijento Kyūbu), also known as Intelligent Qube in North America and Kurushi in Europe, is a puzzle video game for the PlayStation. In the game, the player controls a character who must run around a platform made of cubes, clearing certain cubes as they approach. Cubes are "cleared" by marking a spot on the stage, waiting for the cube to roll on top of it, and then deactivating the marked spot.
The game was well received by critics. The game performed well commercially in Japan and even won the Excellence Award for Interactive Art at the 1997 Japan Media Arts Festival.

Page: Spin qubit quantum computer
Summary: The spin qubit quantum computer is a quantum computer based on controlling the spin of charge carriers (electrons and electron holes) in semiconductor devices. The first spin qubit quant

{'input': 'What is Q* algorithm?',
 'output': 'I found a few references to "Q* algorithm" but none of them seem to be directly related to a specific algorithm. There is a puzzle video game called "I.Q.: Intelligent Qube" and a quantum computer concept called the "Spin qubit quantum computer" which was first proposed by Daniel Loss and David P. DiVincenzo in 1997. There is also a positional 3D sound processing algorithm called "QSound". However, it\'s possible that "Q* algorithm" refers to something else that I couldn\'t find information on.'}

In [67]:
agent_executor.invoke({"input": "Hello I am Abhishek"})



> Entering new AgentExecutor chain...
Hello Abhishek! How can I assist you today?

> Finished chain.


{'input': 'Hello I am Abhishek',
 'output': 'Hello Abhishek! How can I assist you today?'}

In [68]:
agent_executor.invoke({"input": "Can you tell me my name?"})



> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to personal information like your name.

> Finished chain.


{'input': 'Can you tell me my name?',
 'output': "I'm sorry, but I don't have access to personal information like your name."}

In [69]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful kind assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [70]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [71]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

In [72]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, memory=memory, verbose=True)

In [73]:
agent_executor.invoke({
    "input": "Hello! I am Abhishek"
})



> Entering new AgentExecutor chain...
Hello Abhishek! How can I assist you today?

> Finished chain.


{'input': 'Hello! I am Abhishek',
 'chat_history': [HumanMessage(content='Hello! I am Abhishek'),
  AIMessage(content='Hello Abhishek! How can I assist you today?')],
 'output': 'Hello Abhishek! How can I assist you today?'}

In [74]:
agent_executor.invoke({
    "input": "Do you remember my name?"
})



> Entering new AgentExecutor chain...
Yes, your name is Abhishek. How can I assist you today, Abhishek?

> Finished chain.


{'input': 'Do you remember my name?',
 'chat_history': [HumanMessage(content='Hello! I am Abhishek'),
  AIMessage(content='Hello Abhishek! How can I assist you today?'),
  HumanMessage(content='Do you remember my name?'),
  AIMessage(content='Yes, your name is Abhishek. How can I assist you today, Abhishek?')],
 'output': 'Yes, your name is Abhishek. How can I assist you today, Abhishek?'}

### Create a chatbot

In [76]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [77]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [78]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [79]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# QnA_Bot')),
    pn.Tabs(('Conversation', tab1))
)
dashboard

Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, height=400, loading_indicator=True)
            [3] Divider()